In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import glob, json, os, sys

sys.path.append('/mnt/llm/home/ibragim-bad/code/SWE-bench/swebench/metrics')
sys.path.append('/mnt/llm/home/ibragim-bad/code/SWE-bench/') # TODO: Replace with path to `SWE-bench/swebench/metrics` folder
from conversion import convert_log_to_ground_truth
from getters import get_logs_gold
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

sys.path.append('/mnt/llm/home/ibragim-bad/code/SWE-bench/swebench/harness') # TODO: Replace with path to `SWE-bench/swebench/harness` folder
from utils import has_attribute_or_import_error
sys.path = sys.path[:-1]

/mnt/llm/home/ibragim-bad/code/conda/envs/swe-bench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Declare repository; Fetch tasks, logs

In [3]:
repo = 'Farama-Foundation/Minigrid' # TODO: Replace with repository name
log_dir = '/mnt/llm/home/ibragim-bad/code/SWE-bench/data/log_dir' # TODO: Replace with path to folder of execution logs

Get map of version to setup commit

In [28]:
tasks = json.load(open('/mnt/llm/home/ibragim-bad/code/SWE-bench/data/testbed/versions/Minigrid-task-instances_versions.json')) # TODO: Replace with path to versioned candidate task instances
tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)
version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

#### Monitor Validation

In [29]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

Total Attempts: 65
Failed: 56
Usable: 9
Corrupt Test: 1
Corrupt Diff: 0
Test Script Timeout: 0
Success E2E: 8


In [30]:
logs_same, logs_diff = monitor_logs_same_diff(log_dir)
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

Logs same: 4
Logs diff: 4


#### Get [FP]2[FP] Tests

In [31]:
inst_id_to_gt = {}
for d in logs_diff:
    status_gt = convert_log_to_ground_truth(d[0])
    inst_id_to_gt[d[0]] = status_gt

#### Create Task Instances `.json` file

In [32]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    if len(v['FAIL_TO_PASS']) == 0:
        continue
    if get_id_from_log(k) not in tasks:
        print(get_id_from_log(k))
        continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit[task['version']]

    # Do not consider tasks where the log before the patch has an attribute/import error
    log_path = os.path.join(log_dir, f'{task["instance_id"]}.log')
    log_before, log_after = get_logs_gold(log_path)
    if has_attribute_or_import_error(log_before):
        continue

    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))

SAVE_PATH = "/mnt/llm/home/ibragim-bad/code/SWE-bench/data/minigrid_results.json" # TODO: Replace this with a path to a .json file to save the task instances
with open(SAVE_PATH, 'w') as f:
    json.dump(tasks_final, fp=f)

SheffieldML__GPy-1070
Final number of task instances:  3


In [33]:
import pandas as pd 

pd.DataFrame(tasks_final)

,repo,pull_number,instance_id,issue_numbers,base_commit,patch,test_patch,problem_statement,hints_text,created_at,version,FAIL_TO_PASS,PASS_TO_PASS,environment_setup_commit
0,Farama-Foundation/Minigrid,304,Farama-Foundation__Minigrid-304,[303],b7d447b34fc05565b05a28304e5b981a0a714b05,diff --git a/minigrid/core/mission.py b/minigr...,diff --git a/tests/test_envs.py b/tests/test_e...,[Bug Report] Vectorizing minigrid environments...,,2023-02-09T10:25:49Z,2.1,[tests/test_envs.py::test_env_sync_vectorizati...,[tests/test_envs.py::test_env[MiniGrid-Blocked...,488b25db0df78dee21447314429892c82a069b5e
1,Farama-Foundation/Minigrid,331,Farama-Foundation__Minigrid-331,[327],488b25db0df78dee21447314429892c82a069b5e,diff --git a/minigrid/wrappers.py b/minigrid/w...,diff --git a/tests/test_wrappers.py b/tests/te...,[Question] Is SymbolicObsWrapper wrong in Dist...,,2023-03-13T16:21:50Z,2.1,[tests/test_wrappers.py::test_symbolic_obs_wra...,[tests/test_wrappers.py::test_reseed_wrapper[M...,488b25db0df78dee21447314429892c82a069b5e
2,Farama-Foundation/Minigrid,381,Farama-Foundation__Minigrid-381,[308],546c040a9424f339b7ba69e911e4126cee216e64,diff --git a/minigrid/envs/babyai/putnext.py b...,diff --git a/tests/test_baby_ai_bot.py b/tests...,[Question] Any plan to support BabyAI bot ?\n#...,I haven't done much with BabyAI but I would ad...,2023-07-19T09:19:35Z,2.3,[tests/test_baby_ai_bot.py::test_bot[BabyAI-Go...,[],546c040a9424f339b7ba69e911e4126cee216e64


In [27]:
from copy import deepcopy